In [ ]:
### INSTALL CASADI ####

!pip install casadi

In [11]:
import casadi as ca
import numpy as np
import time
import math

v_e = 0.75
v_p = 0.75
N = 20
dt = 1/N


opti = ca.Opti()

# constraint lambda, mu
lam = opti.variable(3, N)  # index = 1,...,T
mu = opti.variable(1, N)
mu_bar = opti.variable(1, N)
epsilon = opti.variable(1, N)
C = opti.variable(1, 1)
action = opti.variable(1, 1)

# control variable
u = opti.variable(1, N)
d = opti.variable(1, N)


# state
states = opti.variable(3, N)
x1 = states[0, :]
x2 = states[1, :]
x3 = states[2, :]

# parameters
opt_x0 = opti.parameter(3)

# dynamic function
def f(x_, u_, d_): return ca.vertcat(
     -0.75 + 0.75 * ca.cos(x_[2]) + u_ * ca.sin(x_[1]),
      0.75 * ca.sin(x_[2]) - u_ * x_[0],
      d_ - u_)

def l(x_): return ca.sqrt(x_[0]**2 + x_[1]**2 + 1e-9) - 0.25

# gradient l(x_i) w.r.t x_i
def dldx(x_): return ca.vertcat(
      x_[0]/ca.sqrt(x_[0]**2 + x_[1]**2 + 1e-9),
      x_[1]/ca.sqrt(x_[0]**2 + x_[1]**2 + 1e-9),
      0)

# gradient f(x_i) w.r.t x_i, I + df/dx * dt
def dfdx(x_, u_, dt):
    row1 = ca.horzcat(1, -u_*dt, 0)
    row2 = ca.horzcat(u_*dt, 1, 0)
    row3 = ca.horzcat(-0.75*ca.sin(x_[2])*dt, 0.75*ca.cos(x_[2])*dt, 1)
    return ca.vertcat(row1,
                      row2,
                      row3)

def u_star(lam, x_, dt):     
    return 3*ca.sign((lam[0]*x_[1] - lam[1]*x_[0]-lam[2])*dt)
    
def d_star(lam, x_, dt):return -3*ca.sign(lam[2]*dt)
    

# init_condition
opti.subject_to(states[:, 0] == opt_x0)

# dynamic constraint
for i in range(N - 1):
#     u[:, i] = u_star(lam[:, i + 1], states[:, i], dt)
    d[:, i] = d_star(lam[:, i+1], states[:, i], dt)
    x_next = states[:, i] + f(states[:, i], u[:, i], d[:, i]) * dt
    opti.subject_to(states[:, i + 1] == x_next)
    

# costate constraint
for i in range(1, N - 1):
    opti.subject_to(-mu[:, i]*dldx(states[:, i]) +mu_bar[:, i]*dldx(states[:, i]) + lam[:, i] -
                    ca.mtimes(dfdx(states[:, i], u[:, i], dt), lam[:, i + 1]) == 0)
opti.subject_to(-mu[:, N - 1] * dldx(states[:, N - 1]) + mu_bar[:, N - 1] * dldx(states[:, N - 1])+
                lam[:, N - 1] == 0)

# multiplier constraint
sum_mu = 0
for i in range(N):
    sum_mu = sum_mu + mu[:, i] - mu_bar[:, i]
    opti.subject_to(mu[:, i] * (C - l(states[:, i])) == 0)
    opti.subject_to(mu_bar[:, i] * (-C + l(states[:, i]) - epsilon[:, i]) == 0)
    opti.subject_to(epsilon[:, i] >= 0)
    opti.subject_to(mu[:, i] >= 0)
    opti.subject_to(mu_bar[:, i] >= 0)
    opti.subject_to(C <= l(states[:, i]))
    opti.subject_to(C >= l(states[:, i]) - epsilon[:, i])
opti.subject_to(sum_mu + 1 == 0)

### INITIAL GUESSES
x1_gus = [0.5, 0.438424627447422, 0.369074705799447, 0.29294769599604625, 0.21113852198328104, 0.12482378011150635, 0.03524493776393854, -0.056309590777699224,
          -0.14852303381775453, -0.2400691394971757, -0.329631192516832, -0.4159210431850402, -0.49769762057167954, -0.573784761288222, -0.6430881186362402,
          -0.7046108937059189, -0.7574682498048215, -0.8008999310771989, -0.8342813035506601, -0.8571322634927412]
x2_gus = [-0.5, -0.5686425857384736, -0.6294201771263511, -0.6814586788587873, -0.7240096128136165, -0.7564610360800551, -0.7783461303099054, -0.7893501572182505,
          -0.789314861416943, -0.7782407556233938, -0.7562870806846811, -0.7237696445279084, -0.6811561073844271, -0.629059335515344, -0.5682286382237293,
          -0.49953898740324815, -0.4239782569058928, -0.3426332533336725, -0.25667390796571504, -0.16733656575476247]
x3_gus = [1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863, 1.5863]


opti.set_initial(x1, x1_gus)
opti.set_initial(x2, x2_gus)
opti.set_initial(x3, x3_gus)

# objective function
obj = -C
for i in range(N):
    obj = obj + epsilon[:, i]

opti.minimize(obj)

# boundrary and control conditions
opti.subject_to(opti.bounded(-1.0, x1, 1.0))
opti.subject_to(opti.bounded(-1.0, x2, 1.0))
opti.subject_to(opti.bounded(-math.pi, x3, math.pi))

opti.subject_to(opti.bounded(-3, u, 3))

ini_state = np.array([[0.5], [-0.5], [1.59]])
opti.set_value(opt_x0, ini_state)

opts_setting = {'ipopt.hessian_approximation':"limited-memory", 'ipopt.max_iter': 15000, 'ipopt.print_level': 5, 'print_time': 0,
                'ipopt.acceptable_tol': 1e-8, 'ipopt.acceptable_obj_change_tol': 1e-6, 'ipopt.inf_pr_output':"original"}

opti.solver('ipopt', opts_setting)
sol = opti.solve()

res_x = sol.value(states)
res_u = sol.value(u)
res_d = sol.value(d)
res_lambda = sol.value(lam)
res_mu = sol.value(mu)
res_epsilon = sol.value(epsilon)

print(res_x)

print()




This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      952
Number of nonzeros in inequality constraint Jacobian.:      280
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      221
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      158
Total number of inequality constraints...............:      180
        inequality constraints with only lower bounds:       60
   inequality constraints with lower and upper bounds:       80
        inequality constraints with only upper bounds:       40

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

 108  3.1280165e+00 1.55e-03 2.83e+05  -1.9 3.08e+01    -  2.25e-01 1.94e-02f  1
 109  3.0659443e+00 7.12e-03 1.53e+05  -2.8 1.99e+00    -  8.00e-01 5.23e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 110  3.0646611e+00 3.63e-03 6.34e+04  -3.4 3.79e-01    -  9.75e-01 5.00e-01h  2
 111  3.0637457e+00 1.66e-03 3.37e+04  -4.2 1.52e-02    -  1.00e+00 5.40e-01h  1
 112  3.0611948e+00 4.33e-04 4.31e+04  -4.5 2.36e-02    -  1.00e+00 7.39e-01h  1
 113  3.0589986e+00 3.68e-04 1.01e+05  -4.9 2.89e-01    -  5.31e-01 1.63e-01h  1
 114  3.0409211e+00 3.00e-01 9.96e+04  -3.9 1.83e+01    -  2.22e-02 2.11e-02f  1
 115  3.0322972e+00 2.70e-01 8.62e+04  -3.8 2.57e+00    -  1.67e-01 1.00e-01h  1
 116  3.0260014e+00 6.83e-02 7.34e+04  -3.7 1.54e+00    -  2.44e-01 1.42e-01h  1
 117  3.0211132e+00 5.02e-02 5.28e+04 -10.1 8.55e-01    -  1.93e-01 2.65e-01h  1
 118  3.0211138e+00 4.82e-02 2.70e+04  -3.8 3.93e-01    -  4.01e-01 4.07e-02h  1
 119  3.0191129e+00 2.61e-01

 268  3.0242333e+00 2.69e-01 1.64e+06  -3.6 4.05e-02    -  3.73e-01 2.50e-01h  3
 269  3.0235852e+00 1.04e-01 1.21e+06  -3.6 2.58e-01    -  2.20e-01 2.72e-01w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 270  3.0231043e+00 6.33e-02 7.51e+05  -3.6 1.03e-01    -  4.28e-01 3.91e-01w  1
 271  3.0235128e+00 5.13e-02 5.92e+05  -3.6 5.69e-02    -  9.13e-01 1.89e-01w  1
 272  3.0241523e+00 2.60e-01 1.59e+06  -3.6 4.94e-02    -  2.20e-01 3.40e-02h  3
 273  3.0236595e+00 5.17e-02 1.50e+06  -3.6 2.36e-01    -  8.64e-01 4.61e-02h  2
 274  3.0170416e+00 2.62e-01 2.72e+05  -3.6 5.12e-02    -  1.00e+00 8.49e-01H  1
 275  3.0112005e+00 3.00e-01 2.50e+04  -3.6 2.44e-01    -  6.14e-01 1.00e+00H  1
 276  3.0218132e+00 7.73e-02 9.86e+03  -4.0 2.72e-01    -  9.29e-01 7.42e-01h  1
 277  3.0185718e+00 1.79e-01 1.42e+04  -4.1 7.43e-02    -  9.36e-01 4.58e-01H  1
 278  3.0155067e+00 3.00e-01 1.49e+04  -4.1 1.68e-01    -  1.00e+00 4.79e-01F  1
 279  3.0174533e+00 2.25e-01

 414r 3.0854480e+00 4.46e-02 1.00e+03  -1.4 0.00e+00    -  0.00e+00 2.83e-07R 16
 415r 3.0875033e+00 3.00e-01 1.48e+03  -0.1 4.05e+01    -  1.63e-01 1.09e-03f  1
 416r 3.0925908e+00 2.54e-01 9.65e+02  -1.9 5.62e-01    -  3.93e-01 1.55e-01f  1
 417r 3.0993654e+00 1.39e-01 2.89e+02  -2.7 2.35e-01    -  8.47e-01 4.53e-01f  1
 418r 3.1102996e+00 2.91e-01 1.94e+01  -3.2 1.30e-01    -  9.11e-01 9.33e-01f  1
 419r 3.1155889e+00 2.70e-01 1.24e+02  -3.2 2.70e-01    -  6.02e-01 8.97e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 420r 3.1618191e+00 2.79e-01 4.13e+01  -3.7 2.66e-01    -  8.12e-01 9.22e-01h  1
 421r 3.1218393e+00 2.28e-01 1.90e+01  -4.0 3.62e-01    -  7.86e-01 7.41e-01h  1
 422r 3.1505962e+00 1.11e-01 2.53e+02  -4.0 2.31e-01    -  1.00e+00 5.11e-01h  1
 423r 3.1428655e+00 2.55e-01 6.34e+01  -5.1 1.04e-01    -  9.98e-01 5.97e-01h  1
 424r 3.1464209e+00 3.00e-01 7.72e+00  -4.8 2.40e-01    -  9.02e-01 9.12e-01h  1
 425r 3.1396420e+00 3.00e-01

 560  5.2410276e+00 5.51e-02 5.26e+06  -1.7 5.24e-02    -  3.70e-01 3.71e-03h  9
 561  5.2441780e+00 2.48e-01 5.63e+06  -1.7 5.23e-02    -  2.85e-01 6.11e-02h  5
 562  5.2456637e+00 5.95e-02 8.14e+06  -1.7 2.32e-01    -  1.00e+00 3.12e-02h  6
 563  5.2485899e+00 5.58e-02 1.05e+07  -1.7 5.64e-02    -  1.00e+00 6.25e-02h  5
 564  5.2492994e+00 5.49e-02 1.16e+07  -1.7 5.24e-02    -  1.00e+00 1.56e-02h  7
 565  5.2493054e+00 5.49e-02 1.26e+07  -1.7 5.16e-02    -  1.00e+00 1.22e-04h 14
 566  5.2493091e+00 5.49e-02 1.34e+07  -1.7 5.16e-02    -  1.00e+00 6.10e-05h 15
 567  5.3157674e+00 3.00e-01 2.66e+05  -1.7 5.16e-02    -  1.00e+00 1.00e+00w  1
 568  3.3701863e+00 7.95e-02 1.20e+07  -1.7 3.50e-01    -  8.98e-01 2.65e-01h  1
 569  3.3932199e+00 5.92e-02 8.99e+06  -1.7 6.93e-02    -  2.04e-01 2.56e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 570  3.3919120e+00 5.56e-02 8.95e+06  -1.7 3.68e-01    -  3.10e-03 6.00e-02f  1
 571  3.4466885e+00 5.39e-02

...

14643r 4.8818855e+00 6.92e-02 1.28e+01  -2.9 5.52e-02    -  1.00e+00 4.88e-04h 12
14644r 4.8818853e+00 6.92e-02 1.95e+01  -2.9 5.59e-02    -  1.00e+00 1.22e-04h 14
14645r 4.8818852e+00 6.92e-02 2.15e+01  -2.9 5.41e-02    -  1.00e+00 6.10e-05h 15
14646r 4.8818852e+00 6.92e-02 2.52e+01  -2.9 1.31e-01    -  1.00e+00 1.53e-05h 17
14647r 4.8814062e+00 3.00e-01 1.83e+01  -2.9 7.67e-02    -  1.00e+00 1.00e+00s 22
14648r 4.8814062e+00 3.00e-01 9.66e+00  -2.9 3.63e-01    -  0.00e+00 0.00e+00R  1
14649r 4.8805778e+00 2.28e-01 2.47e+02  -2.9 3.63e-01    -  1.00e+00 2.41e-01f  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
14650r 4.8805620e+00 2.24e-01 5.66e+01  -2.9 2.28e-01    -  1.00e+00 1.48e-02f  7
14651r 4.8805538e+00 2.24e-01 6.20e+01  -2.9 2.24e-01    -  1.00e+00 1.95e-03f 10
14652r 4.8805531e+00 2.24e-01 4.46e+01  -2.9 2.24e-01    -  1.00e+00 2.44e-04f 13
14653r 4.8805529e+00 2.24e-01 4.22e+01  -2.9 2.24e-01    -  1.00e+00 6.10e-05f 15
14654r 4.8794813e

14738r 4.8698636e+00 2.38e-01 2.40e+02  -2.9 5.00e-01    -  1.00e+00 3.68e-01h  2
14739r 4.8884065e+00 3.00e-01 2.79e+01  -2.9 2.56e-01    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
14740r 4.8862086e+00 3.00e-01 2.11e+00  -2.9 2.82e-01    -  1.00e+00 1.00e+00h  1
14741r 4.8837494e+00 1.50e-01 8.05e+01  -2.9 5.49e-01    -  1.00e+00 5.00e-01h  2
14742r 4.8865533e+00 3.00e-01 8.72e+01  -2.9 2.48e-01    -  1.00e+00 1.00e+00h  1
14743r 4.8795003e+00 3.00e-01 2.58e+02  -2.9 7.47e-01    -  9.69e-01 5.04e-01h  1
14744r 4.8615037e+00 1.50e-01 1.14e+02  -2.9 3.97e-01    -  5.42e-01 5.00e-01h  2
14745r 4.8616693e+00 1.41e-01 1.54e+02  -2.9 7.78e-02    -  1.00e+00 6.25e-02h  5
14746r 4.8618732e+00 1.36e-01 2.77e+02  -2.9 7.44e-02    -  1.00e+00 3.12e-02h  6
14747r 4.8619070e+00 1.36e-01 2.94e+02  -2.9 8.49e-02    -  1.00e+00 1.95e-03h 10
14748r 4.8619221e+00 1.36e-01 2.97e+02  -2.9 7.19e-02    -  1.00e+00 9.77e-04h 11
14749r 4.8619255e

14847r 4.8757213e+00 7.15e-02 2.72e+01  -2.9 6.47e-02    -  1.00e+00 3.12e-02h  6
14848r 4.8757504e+00 7.04e-02 3.36e+01  -2.9 6.23e-02    -  1.00e+00 1.56e-02h  7
14849r 4.8757650e+00 6.99e-02 3.08e+01  -2.9 9.58e-02    -  1.00e+00 7.81e-03h  8
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
14850r 4.8757731e+00 6.96e-02 2.74e+01  -2.9 6.72e-02    -  1.00e+00 3.91e-03h  9
14851r 4.8757754e+00 6.95e-02 2.86e+01  -2.9 8.00e-02    -  1.00e+00 9.77e-04h 11
14852r 4.8757765e+00 6.95e-02 2.82e+01  -2.9 8.83e-02    -  1.00e+00 4.88e-04h 12
14853r 4.8780671e+00 3.00e-01 6.27e+00  -2.9 7.32e-02    -  1.00e+00 1.00e+00w  1
14854r 4.8745296e+00 3.00e-01 1.79e+01  -2.9 2.74e-01    -  1.00e+00 1.00e+00w  1
14855r 4.8786111e+00 3.00e-01 1.73e+01  -2.9 2.89e-01    -  1.00e+00 1.00e+00w  1
14856r 4.8757770e+00 6.95e-02 2.78e+01  -2.9 5.82e-01    -  1.00e+00 2.44e-04h 12
14857r 4.8757771e+00 6.95e-02 2.84e+01  -2.9 6.17e-02    -  1.00e+00 1.22e-04h 14
14858r 4.8766030e

14942r 4.7879302e+00 1.06e-03 7.58e+01  -3.0 3.99e-01    -  1.00e+00 1.56e-02h  7
14943r 4.7880143e+00 1.05e-03 7.93e+01  -3.0 4.03e-01    -  1.00e+00 7.81e-03h  8
14944r 4.7880144e+00 1.05e-03 7.25e+01  -3.0 3.57e-01    -  1.00e+00 1.53e-05h 17
14945r 4.7872216e+00 3.00e-01 1.45e+00  -3.0 2.56e-03    -  1.00e+00 1.00e+00h  1
14946r 4.7864947e+00 3.00e-01 1.83e+00  -3.0 2.81e-01    -  1.00e+00 1.00e+00h  1
14947r 4.7610979e+00 3.00e-01 1.04e+02  -3.0 2.84e-01    -  1.00e+00 1.00e+00H  1
14948r 4.7800477e+00 3.00e-01 2.06e+02  -3.0 7.46e-01    -  1.00e+00 4.87e-01h  2
14949r 4.7670716e+00 2.46e-01 1.54e+02  -3.0 5.84e-01    -  1.00e+00 6.32e-01H  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
14950r 4.7723741e+00 1.54e-01 1.45e+02  -3.0 4.54e-01    -  1.00e+00 3.75e-01h  2
14951r 4.7738183e+00 1.36e-01 3.79e+02  -3.0 2.70e+00    -  4.35e-01 1.15e-01h  4
14952r 4.7798666e+00 6.81e-02 1.24e+02  -3.0 2.49e-01    -  1.00e+00 5.00e-01h  2
14953r 4.7799158e

RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:159:
.../casadi/core/optistack_internal.cpp:999: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Maximum_Iterations_Exceeded'